In [1]:
import cv2
import argparse
import numpy as np
f
from tqdm import tqdm
from loguru import logger
from ultralytics import YOLO
from collections import defaultdict

In [ ]:
def load_config():
    return {
        "model_path": "yolo11x.pt",
        "track_history_length": 120,
        "batch_size": 64,
        "line_thickness": 4,
        "track_color": (230, 230, 230)
    }
    
def inititalize_video(video_path):
    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    video_name = video_path.split("/")[-1]
    output_path = f"run/{video_name.split('.')[0]}_tracked.mp4"
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    return cap, out, output_path

In [ ]:
def update_track_history(
    track_history, last_seen, track_ids, frame_count, batch_size, frame_idx, history_length
):
    current_tracks = set(track_ids)
    for track_id in list(track_history.keys()):
        if track_id in current_tracks:
            last_seen[track_id] = frame_count - (batch_size - frame_idx - 1)
            
        elif frame_count - last_seen[track_id] > history_length:
            del track_history[track_id]
            del last_seen[track_id]

In [ ]:
def draw_tracks(frame, boxes, track_ids, track_history, config):
    if not track_ids:
        return frame
    
    for box, track_id in zip(boxes, track_ids):
        x, y, w, h = box
        track = track_history[track_id]
        track.append((float(x), float(y)))
        if len(track) > config["track_history_length"]:
            track.pop(0)
            
        points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
        cv2.polylines(
            frame, [points], isClosed=False,
            color=config["track_color"], thickness=config["line_thickness"]
        )
    
    return frame